# Ejercicio2, Redes Neuronales EMNIST

Nombre: Quispe Taboada Daniel <br>
Dataset: ['EMNIST'](https://www.kaggle.com/datasets/keshavsharma2/emnist)

In [144]:
import pandas as pd
import numpy as np

In [145]:
def displayData(X, example_width=None, figsize=(10, 10)):
    """
    Muestra datos 2D almacenados en X en una cuadrícula apropiada.
    """
    # Calcula filas, columnas
    if X.ndim == 2:
        m, n = X.shape
    elif X.ndim == 1:
        n = X.size
        m = 1
        X = X[None]  # Promocionar a una matriz bidimensional
    else:
        raise IndexError('La entrada X debe ser 1 o 2 dimensinal.')

    example_width = example_width or int(np.round(np.sqrt(n)))
    example_height = n / example_width

    # Calcula el numero de elementos a mostrar
    display_rows = int(np.floor(np.sqrt(m)))
    display_cols = int(np.ceil(m / display_rows))

    fig, ax_array = pyplot.subplots(display_rows, display_cols, figsize=figsize)
    fig.subplots_adjust(wspace=0.025, hspace=0.025)

    ax_array = [ax_array] if m == 1 else ax_array.ravel()

    for i, ax in enumerate(ax_array):
        ax.imshow(X[i].reshape(example_width, example_width, order='F'),
                  cmap='Greys', extent=[0, 1, 0, 1])
        ax.axis('off')

# El Perceptrón Multicapa - Clasificación

### Funciones de activación

Para la capa oculta de nuestro MLP utilizaremos una función de activación de tipo `relu`, de la cual necesitaremos su derivada.

In [146]:
def relu(x):
  return np.maximum(0, x)

def reluPrime(x):
  return x > 0

En cuanto a las funciones de activación que utilizaremos a la salida del MLP, éstas son las que hemos introducido en posts anteriores:

- Lineal: usada para regresión (junto a la función de pérdida MSE).
- Sigmoid: usada para clasificación binaria (junto a la función de pérdida BCE).
- Softmax: usada para clasificación multiclase (junto a la función de pérdida crossentropy, CE).

In [147]:
def linear(x):
    return x

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

### Funciones de pérdida

Como acabamos de comentar en la sección anterior, estas son las funciones de pérdida que hemos visto hasta ahora para las diferentes tareas.

In [148]:
# Mean Square Error -> usada para regresión (con activación lineal)
def mse(y, y_hat):
    return np.mean((y_hat - y.reshape(y_hat.shape))**2)

# Binary Cross Entropy -> usada para clasificación binaria (con sigmoid)
def bce(y, y_hat):
    return - np.mean(y.reshape(y_hat.shape)*np.log(y_hat) - (1 - y.reshape(y_hat.shape))*np.log(1 - y_hat))

# Cross Entropy (aplica softmax + cross entropy de manera estable) -> usada para clasificación multiclase
def crossentropy(y, y_hat):
    logits = y_hat[np.arange(len(y_hat)),y]
    entropy = - logits + np.log(np.sum(np.exp(y_hat),axis=-1))
    return entropy.mean()

Y sus derivadas

In [149]:
def grad_mse(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_bce(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_crossentropy(y, y_hat):
    answers = np.zeros_like(y_hat)
    answers[np.arange(len(y_hat)),y] = 1    
    return (- answers + softmax(y_hat)) / y_hat.shape[0]

### Implementación MLP

Ahora que ya tenemos definidas las diferentes funciones de activación y de pérdida que necesitamos, vamos a implementar nuestro MLP de dos capas capaz de llevar a cabo tanto tareas de regresión como de clasificación. Del mismo modo que ya hicimos con el `Perceptrón`, definiremos una clase base que servirá para la implementación de las clases particulares para cada caso.

In [150]:
# clase base MLP 

class MLP():
  def __init__(self, D_in, H, D_out, loss, grad_loss, activation):
    # pesos de la capa 1
    self.w1, self.b1 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(D_in+H)),
                                  size=(D_in, H)), np.zeros(H)
    # pesos de la capa 2
    self.w2, self.b2 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(H+D_out)),
                                  size=(H, D_out)), np.zeros(D_out)
    self.ws = []
    # función de pérdida y derivada
    self.loss = loss
    self.grad_loss = grad_loss
    # función de activación
    self.activation = activation

  def __call__(self, x):
    # salida de la capa 1
    self.h_pre = np.dot(x, self.w1) + self.b1
    self.h = relu(self.h_pre)
    # salida del MLP
    y_hat = np.dot(self.h, self.w2) + self.b2 
    return self.activation(y_hat)
    
  def fit(self, X, Y, epochs = 100, lr = 0.001, batch_size=None, verbose=True, log_each=1):
    batch_size = len(X) if batch_size == None else batch_size
    batches = len(X) // batch_size
    l = []
    for e in range(1,epochs+1):     
        # Mini-Batch Gradient Descent
        _l = []
        for b in range(batches):
            # batch de datos
            x = X[b*batch_size:(b+1)*batch_size]
            y = Y[b*batch_size:(b+1)*batch_size] 
            # salida del perceptrón
            y_pred = self(x) 
            # función de pérdida
            loss = self.loss(y, y_pred)
            _l.append(loss)        
            # Backprop 
            dldy = self.grad_loss(y, y_pred) 
            grad_w2 = np.dot(self.h.T, dldy)
            grad_b2 = dldy.mean(axis=0)
            dldh = np.dot(dldy, self.w2.T)*reluPrime(self.h_pre)      
            grad_w1 = np.dot(x.T, dldh)
            grad_b1 = dldh.mean(axis=0)
            # Update (GD)
            self.w1 = self.w1 - lr * grad_w1
            self.b1 = self.b1 - lr * grad_b1
            self.w2 = self.w2 - lr * grad_w2
            self.b2 = self.b2 - lr * grad_b2
        l.append(np.mean(_l))
        # guardamos pesos intermedios para visualización
        self.ws.append((
            self.w1.copy(),
            self.b1.copy(),
            self.w2.copy(),
            self.b2.copy()
        ))
        if verbose and not e % log_each:
            print(f'Epoch: {e}/{epochs}, Loss: {np.mean(l):.5f}')

  def predict(self, ws, x):
    w1, b1, w2, b2 = ws
    h = relu(np.dot(x, w1) + b1)
    y_hat = np.dot(h, w2) + b2
    return self.activation(y_hat)

In [151]:
# MLP para regresión
class MLPRegression(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, mse, grad_mse, linear)

# MLP para clasificación binaria
class MLPBinaryClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, bce, grad_bce, sigmoid)

# MLP para clasificación multiclase
class MLPClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, crossentropy, grad_crossentropy, linear)

Vamos a probar ahora nuestra implementación para diferentes ejemplos.

## Clasificación Multiclase

Por último vamos a ver cómo aplicar nuestro modelo para clasificación en multiples clases.

In [152]:
# Para trabajar en colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [153]:
# para colab
# dataN = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SegundoPRacial/Data/processed_balanced_test.csv", sep=',')
dataN = pd.read_csv("Data/processed_balanced_test.csv", sep=',')
data = dataN.to_numpy()

In [154]:
dataN.shape

(20800, 785)

In [155]:
X, y = data[:, 1:20000], data[:, 0]
salidas = len(np.unique(y))
m, n = X.shape

In [156]:
# X_mean, X_std = X.mean(axis=0), X.std(axis=0)
# X_norm = (X - X_mean) / X_std
# X_norm = X/255

In [157]:
def controlador(data):
    listaMayor = []
    for value in data:
        mayor = 0
        for index in range( len(value) ):
            if( value[index] >= value[mayor]):
                mayor = index
        listaMayor.append(mayor)
    return listaMayor

In [161]:
model = MLPClassification(D_in=n, H=200, D_out=salidas)
epochs, lr = 500, 0.8
model.fit(X, y, epochs, lr, batch_size=100, log_each=10)

Epoch: 10/500, Loss: 1.11027
Epoch: 20/500, Loss: 0.72752
Epoch: 30/500, Loss: 0.53476
Epoch: 40/500, Loss: 0.41682
Epoch: 50/500, Loss: 0.33884
Epoch: 60/500, Loss: 0.28484
Epoch: 70/500, Loss: 0.24547
Epoch: 80/500, Loss: 0.21557
Epoch: 90/500, Loss: 0.19213
Epoch: 100/500, Loss: 0.17328
Epoch: 110/500, Loss: 0.15780
Epoch: 120/500, Loss: 0.14486
Epoch: 130/500, Loss: 0.13389
Epoch: 140/500, Loss: 0.12447
Epoch: 150/500, Loss: 0.11628
Epoch: 160/500, Loss: 0.10912
Epoch: 170/500, Loss: 0.10278
Epoch: 180/500, Loss: 0.09715
Epoch: 190/500, Loss: 0.09210
Epoch: 200/500, Loss: 0.08755
Epoch: 210/500, Loss: 0.08343
Epoch: 220/500, Loss: 0.07969
Epoch: 230/500, Loss: 0.07627
Epoch: 240/500, Loss: 0.07313
Epoch: 250/500, Loss: 0.07024
Epoch: 260/500, Loss: 0.06757
Epoch: 270/500, Loss: 0.06510
Epoch: 280/500, Loss: 0.06280
Epoch: 290/500, Loss: 0.06066
Epoch: 300/500, Loss: 0.05866
Epoch: 310/500, Loss: 0.05679
Epoch: 320/500, Loss: 0.05504
Epoch: 330/500, Loss: 0.05339
Epoch: 340/500, Los

In [164]:
input_layer_size  = 784
numero = 20500
XPrueba = X[numero:numero+100, :].copy()
peso = [model.w1, model.b1, model.w2, model.b2]
prediccion = model.predict(peso, XPrueba)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(controlador(prediccion) == y[numero:numero+100]) * 100))

Precision del conjuto de entrenamiento: 99.00%
